In [2]:
import pandas as pd
import numpy as np
import requests
import json

In [2]:
df = pd.read_csv('上海商圈数据.csv')

In [3]:
df.head()

,cenggao,floor,function_list,jianzhumianji,jinshen,location,loupandizhi,loupanmingcheng,miankuan,release_date,service_price,shangpu_type,shifoufenge,shihejingying,titile,total_price,url,zhuangxiu,shiyongmianji
0,4.50米,第2层(共2层),"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,燃气",1000㎡,15.00米,"青浦,徐泾,绿地世界中心",绿地世界中心,绿地世界中心,60.00米,2019/8/24 12:42:53,0.00元/㎡·月,写字楼配套底商,否,百货超市,2/10/17号地铁虹桥国家会展中心，虹桥世界中心,4500万,https://sh.shop.fang.com/shou/3_360333589.html,简装修,NaN
1,5.00米,第1层(共1层),"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",82.6㎡,10.00米,"嘉定,南翔,南翔老街",南翔老街,南翔老街,8.00米,2019/8/16 11:42:25,3.00元/㎡·月,临街门面,否,"百货超市,生活服务,美容美发,餐饮美食,休闲娱乐","南翔老街入口（全家）年租25万,重餐饮,老板回笼资金，给钱就卖！",258万,https://sh.shop.fang.com/shou/3_359479506.html,精装修,NaN
2,4.50米,NaN,"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",98㎡,9.00米,"徐汇,徐家汇,天钥桥路",天钥桥路,天钥桥路,10.00米,2019/8/14 11:13:56,5.00元/㎡·月,商业街商铺,否,"美容美发,餐饮美食,服饰鞋包,生活服务,酒店宾馆,家居建材,百货超市,休闲娱乐",9号线地铁口+ 纯沿街+品牌商家+ 来伊份+即买收租+ 年租4房东急售,560万,https://sh.shop.fang.com/shou/3_359235355.html,简装修,90㎡
3,5.20米,第2层(共6层),"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",188.5㎡,16.00米,"徐汇,徐家汇,美罗城",肇嘉浜路1115号,美罗城,12.00米,2019/8/7 11:57:23,25.00元/㎡·月,购物中心/百货,否,"百货超市,服饰鞋包,生活服务,美容美发,餐饮美食,休闲娱乐",9号线 徐家汇 美罗城 转角大开间 星巴克 年租金90万 急售！！！,600万,https://sh.shop.fang.com/shou/3_358547073.html,精装修,NaN
4,4.50米,第1层(共1层),"网络,停车位,上水,下水,可明火,排烟,排污,燃气,外摆区",159.67㎡,14.90米,"长宁,北新泾,北渔路沿街餐饮",长宁北新泾北渔路天山西路,北渔路沿街餐饮,10.00米,2019/8/22 20:16:49,5.00元/㎡·月,临街门面,是,"百货超市,酒店宾馆,家居建材,服饰鞋包,生活服务,美容美发,餐饮美食,休闲娱乐",70年独立产权+北渔路沿街餐饮门面+大开间重餐饮+稳定年租58万,770万,https://sh.shop.fang.com/shou/3_360141795.html,精装修,NaN


In [4]:
lst_lpname = df['loupanmingcheng'].value_counts().index.tolist()

lst_lploc = df['loupandizhi'].value_counts().index.tolist()

In [5]:
city = '上海'
dict_location = {}
My_key = 'f654d644aeae0e1da847d7ca408f4370'
url_pre = 'https://restapi.amap.com/v3/geocode/geo?key={}&address={}&city={}'
counter = 0
for i in range(len(lst_lpname)):
    location = lst_lpname[i]
    url = url_pre.format(My_key, location, city)
    res = requests.get(url).json()
    if res['status'] == '0':
        location = lst_lploc[i]
        url = url_pre.format(My_key, location, city)
        res = requests.get(url).json()
        if res['status'] == '0':
            dict_location[lst_lpname[i]] = np.nan
            counter +=1
        else:
            try: 
                dict_location[lst_lpname[i]] = res['geocodes'][0]['location']
                if counter % 250 == 0:
                    print('one batch(250) processed')
            except:
                dict_location[lst_lpname[i]] = np.nan
            
    else:
        try: 
            dict_location[lst_lpname[i]] = res['geocodes'][0]['location']
            if counter % 250 == 0:
                print('one batch(250) processed')
        except:
            location = lst_lploc[i]
            url = url_pre.format(My_key, location, city)
            res = requests.get(url).json()
            if res['status'] == '0':
                dict_location[lst_lpname[i]] = np.nan
                counter +=1
            else:
                try: 
                    dict_location[lst_lpname[i]] = res['geocodes'][0]['location']
                    if counter % 250 == 0:
                        print('one batch(250) processed')
                except:
                    dict_location[lst_lpname[i]] = np.nan
            
        counter +=1

one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed


In [6]:
dict_location

{'南京东路': '121.488365,31.238874',
 '南京西路': '121.472054,31.233774',
 '淮海中路': '121.479996,31.225143',
 '大宁国际商业广场': '121.452424,31.274751',
 '天钥桥路': '121.445251,31.178673',
 '城隍庙': '121.455143,31.190019',
 '上海新天地': '121.474304,31.218637',
 '正大广场': '121.499737,31.237203',
 '南京东路步行街': '121.481933,31.236632',
 '证大大拇指广场': '121.560936,31.226222',
 '豫园老街': '121.491392,31.227393',
 '中铁临港十七区': '121.481933,31.236632',
 '五角场万达广场': '121.513252,31.301154',
 '上海第一八佰伴': '121.517971,31.228374',
 '宝山万达广场': '121.568379,31.115251',
 '美罗城': '121.440426,31.192860',
 '斜土路': '121.448726,31.192243',
 '龙之梦购物中心': '121.416530,31.219773',
 '北上海商业广场': '121.507751,31.265631',
 '黄金城道': '121.394530,31.194332',
 '江桥万达广场': '121.323276,31.238657',
 '长泰广场': '121.600232,31.206871',
 '上海松江万达广场': '121.242857,31.058450',
 '新华红星国际广场': '121.511899,31.076062',
 '临港宝龙城市广场': '121.815086,30.905759',
 '金桥国际商业广场': '121.579223,31.256943',
 '七宝老街': '121.457798,31.212630',
 '田子坊': '121.468417,31.208032',
 '青浦万达茂': '121.082811,31.133373',


In [7]:
df['location_coords'] = np.nan

counter = 0
for i in lst_lpname:
    counter += 1
    df_temp = df[df['loupanmingcheng'] == i]
    df_temp_index = df_temp.index.tolist()
    df.iloc[df_temp_index,19] = dict_location[i]
    if counter %50 == 0:
        print('A batch(50) is done')

A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done


In [8]:
df['city'] = city

In [9]:
df.to_csv('上海商铺地址数据.csv', encoding = 'utf-8-sig',index = False)

In [10]:
df.head()

,cenggao,floor,function_list,jianzhumianji,jinshen,location,loupandizhi,loupanmingcheng,miankuan,release_date,...,shangpu_type,shifoufenge,shihejingying,titile,total_price,url,zhuangxiu,shiyongmianji,location_coords,city
0,4.50米,第2层(共2层),"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,燃气",1000㎡,15.00米,"青浦,徐泾,绿地世界中心",绿地世界中心,绿地世界中心,60.00米,2019/8/24 12:42:53,...,写字楼配套底商,否,百货超市,2/10/17号地铁虹桥国家会展中心，虹桥世界中心,4500万,https://sh.shop.fang.com/shou/3_360333589.html,简装修,NaN,"121.253231,31.331951",上海
1,5.00米,第1层(共1层),"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",82.6㎡,10.00米,"嘉定,南翔,南翔老街",南翔老街,南翔老街,8.00米,2019/8/16 11:42:25,...,临街门面,否,"百货超市,生活服务,美容美发,餐饮美食,休闲娱乐","南翔老街入口（全家）年租25万,重餐饮,老板回笼资金，给钱就卖！",258万,https://sh.shop.fang.com/shou/3_359479506.html,精装修,NaN,"121.308554,31.292458",上海
2,4.50米,NaN,"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",98㎡,9.00米,"徐汇,徐家汇,天钥桥路",天钥桥路,天钥桥路,10.00米,2019/8/14 11:13:56,...,商业街商铺,否,"美容美发,餐饮美食,服饰鞋包,生活服务,酒店宾馆,家居建材,百货超市,休闲娱乐",9号线地铁口+ 纯沿街+品牌商家+ 来伊份+即买收租+ 年租4房东急售,560万,https://sh.shop.fang.com/shou/3_359235355.html,简装修,90㎡,"121.445251,31.178673",上海
3,5.20米,第2层(共6层),"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",188.5㎡,16.00米,"徐汇,徐家汇,美罗城",肇嘉浜路1115号,美罗城,12.00米,2019/8/7 11:57:23,...,购物中心/百货,否,"百货超市,服饰鞋包,生活服务,美容美发,餐饮美食,休闲娱乐",9号线 徐家汇 美罗城 转角大开间 星巴克 年租金90万 急售！！！,600万,https://sh.shop.fang.com/shou/3_358547073.html,精装修,NaN,"121.440426,31.192860",上海
4,4.50米,第1层(共1层),"网络,停车位,上水,下水,可明火,排烟,排污,燃气,外摆区",159.67㎡,14.90米,"长宁,北新泾,北渔路沿街餐饮",长宁北新泾北渔路天山西路,北渔路沿街餐饮,10.00米,2019/8/22 20:16:49,...,临街门面,是,"百货超市,酒店宾馆,家居建材,服饰鞋包,生活服务,美容美发,餐饮美食,休闲娱乐",70年独立产权+北渔路沿街餐饮门面+大开间重餐饮+稳定年租58万,770万,https://sh.shop.fang.com/shou/3_360141795.html,精装修,NaN,"121.568022,31.134725",上海


NameError: name 'df' is not defined

nick starts time: Fri Sep  6 15:01:18 2019
nick starts time: Fri Sep  6 15:01:21 2019
nick starts time: Fri Sep  6 15:01:24 2019
nick starts time: Fri Sep  6 15:01:27 2019
nick starts time: Fri Sep  6 15:01:30 2019
nick starts time: Fri Sep  6 15:01:33 2019
nick starts time: Fri Sep  6 15:01:36 2019
nick starts time: Fri Sep  6 15:01:39 2019
nick starts time: Fri Sep  6 15:01:42 2019
nick starts time: Fri Sep  6 15:01:45 2019
nick starts time: Fri Sep  6 15:01:48 2019
nick starts time: Fri Sep  6 15:01:51 2019
nick starts time: Fri Sep  6 15:01:54 2019
